In [ ]:
!nvidia-smi

Wed Jul 30 15:34:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P8             12W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 46.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.54.0
    Uninstalling transformers-4.54.0:
      Successfully uninstalled transformers-4.54.0


## Local Inference on GPU
Model page: https://huggingface.co/Derify/ChemBERTa_augmented_pubchem_13m

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/Derify/ChemBERTa_augmented_pubchem_13m)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="Derify/ChemBERTa_augmented_pubchem_13m")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/713 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/176M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt:   0%|          | 0.00/14.0 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("Derify/ChemBERTa_augmented_pubchem_13m")
model = AutoModelForMaskedLM.from_pretrained("Derify/ChemBERTa_augmented_pubchem_13m")

#Script

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


code: 3fc59300a8d48949b2cea9ac9c11ba4a017164f5

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import torch
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the ChemBERTa model and tokenizer
model_name = "seyonec/ChemBERTa-zinc-base-v1"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=1)

# Load data
train_df = pd.read_csv('/content/drive/MyDrive/dacon_1/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/dacon_1/test.csv')

# Tokenize SMILES strings with dynamic padding
def tokenize_smiles(smiles):
    return tokenizer(smiles, padding=True, truncation=True, max_length=128, return_tensors='pt')

# Apply tokenization and store as a list
train_tokens = [tokenize_smiles(smiles) for smiles in train_df['Canonical_Smiles']]
test_tokens = [tokenize_smiles(smiles) for smiles in test_df['Canonical_Smiles']]

# No scaling applied to IC50 values
y = train_df['Inhibition'].values

# Split into training and validation sets
train_tokens, val_tokens, y_train, y_val = train_test_split(train_tokens, y, test_size=0.2, random_state=42)

class SMILESDataset(Dataset):
    def __init__(self, tokens, targets=None):
        self.tokens = tokens
        self.targets = targets

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        item = {key: val.squeeze(0) for key, val in self.tokens[idx].items()}
        if self.targets is not None:
            item['labels'] = torch.tensor(self.targets[idx], dtype=torch.float)
        return item

train_dataset = SMILESDataset(train_tokens, y_train)
val_dataset = SMILESDataset(val_tokens, y_val)
test_dataset = SMILESDataset(test_tokens)

# Use DataCollatorWithPadding to handle dynamic padding
data_collator = DataCollatorWithPadding(tokenizer)

# Prepare training arguments
training_args = TrainingArguments(
    output_dir='./',
    eval_strategy="epoch",
    save_strategy="steps",
    save_steps=1000,
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=100,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()

# Predict on the test set
predictions = trainer.predict(test_dataset)

# Save the predictions to a CSV file
output_df = pd.DataFrame({'ID': test_df['ID'], 'Inhibition': predictions.predictions.squeeze()})
output_df.to_csv('./predictions_chembert_v1.csv', index=False)

tokenizer_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/501 [00:00<?, ?B/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kimyeeun293 (ashe13) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,No log,1126.256226
2,No log,1021.742981
3,1177.365300,963.081055
4,1177.365300,920.832520
5,1177.365300,886.755127
6,935.277300,858.185669
7,935.277300,832.390503
8,935.277300,809.882996
9,844.182000,791.492859
10,844.182000,776.016602


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.1

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

def compute_score(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    y_range = y_true.max() - y_true.min()
    norm_rmse = rmse / y_range if y_range != 0 else 0

    if len(np.unique(y_true)) > 1 and len(np.unique(y_pred)) > 1:
        pearson_corr = pearsonr(y_true, y_pred)[0]
    else:
        pearson_corr = 0.0

    score = 0.5 * (1 - min(norm_rmse, 1)) + 0.5 * pearson_corr
    return norm_rmse, pearson_corr, score

val_preds_output = trainer.predict(val_dataset)
val_preds = val_preds_output.predictions.squeeze()

norm_rmse, pearson_corr, combined_score = compute_score(y_val, val_preds)

print(f"Validation Normalized RMSE: {norm_rmse:.6f}")
print(f"Validation Pearson Correlation: {pearson_corr:.6f}")
print(f"Validation Combined Score: {combined_score:.6f}")

Validation Normalized RMSE: 0.273355
Validation Pearson Correlation: 0.280184
Validation Combined Score: 0.503414
